### Paper:
   #### Classification of Malicious Web Code Using Deep Learning [Manoj Kumar Selvam]
### Dataset:
    Total data 11516
    XSS : 7121 + 853 = 7974
    Benign : 3066 + 476 = 3542
   
   

In [144]:
import numpy as np
import pandas as pd
import glob
import time
import pandas as pd
# from xml.dom import minidom

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords 


import os
import matplotlib.pyplot as plt
import keras

[nltk_data] Downloading package punkt to /home/ebryx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ebryx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ebryx/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [145]:
import cv2

In [146]:
path='./data/portswigger.txt'

XSS=[]
f = open(path, "r")
for x in f:
    XSS.append(x)

In [147]:
# Manually removed tag data


path='./data/'
file="scottparker_ml_sqli_src__trainingdata_plain.txt"

In [148]:
df = pd.read_csv(os.path.join(path,file), sep='Aw3s0meSc0t7', names=['benign'], header=None, engine='python')

In [149]:
df.head()

,benign
0,Add plain text here
1,Ne te quaesiveris extra.
2,Man is his own star; and the soul that can
3,"Render an honest and a perfect man,"
4,"Commands all light, all influence, all fate;"


In [150]:
plain_text=df['benign'].values

In [151]:
plain_text[:5]

array(['Add plain text here', 'Ne te quaesiveris extra.',
       'Man is his own star; and the soul that can',
       'Render an honest and a perfect man,',
       'Commands all light, all influence, all fate;'], dtype=object)

In [152]:
plain_text=plain_text[:-22] # removed last 22 records that were urls

In [153]:
len(plain_text)

3641

In [154]:
data=''
for x in plain_text:
    data+=" " + x

In [155]:
stop_words = set(stopwords.words('english')) 
def fun_remove_stop_words(posts):
    filtered=''
    for x in posts.split(' '):
        if x not in stop_words:
            filtered+=' '+x
    return filtered

In [156]:
data=fun_remove_stop_words(data)
data=data.split('.')

In [157]:
data[:5]

['  Add plain text Ne te quaesiveris extra',
 ' Man star; soul Render honest perfect man, Commands light, influence, fate; Nothing falls early late',
 ' Our acts angels are, good ill, Our fatal shadows walk us still',
 ' Epilogue Beaumont Fletcher’s Honest Man’s Fortune Cast bantling rocks, Suckle she-wolf’s teat; Wintered hawk fox, Power speed hands feet',
 ' I read day verses written eminent painter original conventional']

In [158]:
len(data)

3066

In [159]:
## Additional benign data 

path='./data/benign_for_training.txt'
benign_data=[]
f = open(path, "r")
for x in f:
    benign_data.append(x)

In [160]:
path='./data/xss_for_training.txt'
xss_data=[]
f = open(path, "r")
for x in f:
    xss_data.append(x)

In [161]:
values=[]
for i in XSS:
    values.append((i,1))

In [162]:


for i in data:
    values.append((i,0))



In [163]:
for i in benign_data:
    values.append((i,0))

In [164]:
for i in xss_data:
    values.append((i,1))

In [165]:
len(benign_data)

440

In [166]:
len(xss_data)

231

In [167]:
len(XSS)

7121

In [168]:
df=pd.DataFrame(values,columns=['Sentence','Label'])

In [169]:
df.to_csv('XSS_dataset.csv', encoding='utf-8-sig')

In [170]:
len(values)

10858

In [171]:
values[1]

('<a draggable="true" ondragend="alert(1)">test</a>\n', 1)

In [172]:
values[-1]

('document.write()', 1)

In [173]:
sentences=df['Sentence'].values
sentences[1]

'<a draggable="true" ondragend="alert(1)">test</a>\n'

In [174]:
def convert_to_ascii(sentence):
    sentence_ascii=[]

    for i in sentence:
        sentence_ascii.append(ord(i))

    zer=np.zeros((10000))

    for i in range(len(sentence_ascii)):
        zer[i]=sentence_ascii[i]

    zer.shape=(100, 100)


#     plt.plot(image)
#     plt.show()
    return zer


In [175]:
# display_ascii(sentences[100])


arr=np.zeros((10858,100,100))

for i in range(len(sentences)):
    
    image=convert_to_ascii(sentences[i])

    x=np.asarray(image,dtype='float')
    image =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
    image/=128
    
#     if i==1:
#         plt.plot(image)
#         plt.show()    
    arr[i]=image

In [176]:
arr.shape

(10858, 100, 100)

In [177]:
data = arr.reshape(arr.shape[0], 100, 100, 1)

In [178]:
y=df['Label'].values

In [179]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(data,y, test_size=0.2, random_state=42)

In [180]:


from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten,Dropout,MaxPool2D, BatchNormalization



In [181]:

model = Sequential()
model.add(keras.layers.Flatten(input_shape=(100,100,1)))

model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dense(512, activation='relu'))

model.add(keras.layers.Dense(256, activation='relu'))

model.add(keras.layers.Dense(128, activation='relu'))

# model.add(keras.layers.Dense(20,  activation='relu'))
# model.add(keras.layers.Dense(10,  activation='tanh'))
# # model.add(layers.Flatten())
# model.add(keras.layers.Dense(1024, activation='relu'))

model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [182]:


model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()



Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              10241024  
_________________________________________________________________
dense_12 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_13 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_14 (Dense)             (None, 128)               32896     
_________________________________________________________________
batch_normalization_3 (Batch (None, 128)               512       
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

In [183]:
import tensorflow as tf

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.96):
            print(" \n Reached 95% + validation accuracy")
            self.model.stop_training=True
callbacks = myCallback()

In [184]:
batch_size = 128
num_epoch = 10
#model training
model_log = model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=( testX,  testY),
                      callbacks=[callbacks])

Train on 8686 samples, validate on 2172 samples
Epoch 1/10
8686/8686 [==============================] - 18s 2ms/step - loss: 0.3461 - accuracy: 0.8631 - val_loss: 0.4598 - val_accuracy: 0.7136
Epoch 2/10
8686/8686 [==============================] - 18s 2ms/step - loss: 0.1220 - accuracy: 0.9666 - val_loss: 0.2220 - val_accuracy: 0.9185
Epoch 3/10
8686/8686 [==============================] - 17s 2ms/step - loss: 0.0828 - accuracy: 0.9786 - val_loss: 0.1566 - val_accuracy: 0.9567
Epoch 4/10
8686/8686 [==============================] - 18s 2ms/step - loss: 0.0822 - accuracy: 0.9773 - val_loss: 0.1134 - val_accuracy: 0.9613
 
 Reached 95% + validation accuracy


In [185]:
pred=model.predict(testX)

In [186]:
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [187]:
true=0
false=0

for i in range(len(pred)):
    if pred[i] == testY[i]:
        true+=1
    else:
        false+=1

In [188]:
print("correct predicted :: ", true)

correct predicted ::  2088


In [189]:
print("false prediction :: ", false)

false prediction ::  84


In [190]:
attack=0
benign=0
for i in range(len(testY)):
    if testY[i]==1:
        attack+=1
    else:
        benign+=1

In [191]:
print("Attack data in test set :: ", attack)

Attack data in test set ::  1486


In [192]:
print(" Benign data in test set :: ", benign)

 Benign data in test set ::  686


## Confusion Matrix

In [193]:
def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy


In [194]:
def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision


In [195]:


def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall



In [196]:
def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

In [197]:
accuracy,precision,recall=confusion_matrix(testY,pred)

In [198]:
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))


 Accuracy : 0.9613259668508287 
 Precision : 0.9557867360208062 
 Recall : 0.9892328398384926


In [199]:
from sklearn.metrics import precision_score
precision_score(testY, pred)

0.9557867360208062

In [200]:
from sklearn.metrics import recall_score
recall_score(testY, pred)


0.9892328398384926

### Save Model

In [138]:
from keras.models import load_model

model.save('my_model_additional_xss_benign_data.h5')


#### Testing

In [139]:
from keras.models import load_model

mymodel = load_model('my_model_additional_xss_benign_data.h5')

In [201]:
import numpy as np
import cv2
def predict_cross_site_script():
    
    repeat=True
    
    
    
    
    beautify=''
    for i in range(20):
        beautify+= "="

    print(beautify) 
    input_val=input("Give me some data to work on : ")
    print(beautify)
    
    if input_val== '0':
        repeat=False
        
        
    
    if repeat == True:

        sentence_ascii=[]

        for i in input_val:
            sentence_ascii.append(ord(i))

        storage=np.zeros((10000))

        for i in range(len(sentence_ascii)):
            storage[i]=sentence_ascii[i]

        storage.shape=(100, 100)

        x=np.asarray(storage,dtype='float')
        image =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
        image.shape=(1,100,100,1)
        image/=128

        prediction=mymodel.predict(image)

        if prediction>0.5:

    #         print(f"Chances of attack :  {prediction[0]*100} ")
            print(" It can be Cross site scripting attack")

        else:

    #         print(f" Chances of being safe {100 - (prediction*100) }")
            print("seems safe")
        
        
        predict_cross_site_script()
        
        
    elif repeat == False:
        print( " Good Bye ")

In [7]:
# Test for XSS cheatsheet

predict_cross_site_script()

Give me some data to work on : <a id=x tabindex=1 onactivate=alert(1)></a>
 It can be Cross site scripting attack
Give me some data to work on : <body onafterprint=alert(1)>
 It can be Cross site scripting attack
Give me some data to work on : <style>@keyframes x{from {left:0;}to {left: 1000px;}}:target {animation:10s ease-in-out 									0s 1 x;}</style><a id=x style="position:absolute;" onanimationcancel="alert(1)"></a>
 It can be Cross site scripting attack
Give me some data to work on : <style>@keyframes x{}</style><a style="animation-name:x" 									onanimationend="alert(1)"></a>
 It can be Cross site scripting attack
Give me some data to work on : <style>@keyframes slidein {}</style><a style="animation-duration:1s;animation-name:slidein;animation-iteration-count:2" 									onanimationiteration="alert(1)"></a>
 It can be Cross site scripting attack
Give me some data to work on : <style>@keyframes x{}</style><a style="animation-name:x" 									onanimationstart="alert(1)"></a>
 I

 It can be Cross site scripting attack
Give me some data to work on : <article onbeforepaste="alert(1)" contenteditable>test</article>
 It can be Cross site scripting attack
Give me some data to work on : <article onpaste="alert(1)" contenteditable>test</article>
 It can be Cross site scripting attack
Give me some data to work on : <aside draggable="true" ondragleave="alert(1)">test</aside>
 It can be Cross site scripting attack
Give me some data to work on : <aside id=x tabindex=1 onbeforeactivate=alert(1)></aside>
 It can be Cross site scripting attack
Give me some data to work on : <aside onbeforepaste="alert(1)" contenteditable>test</aside>
 It can be Cross site scripting attack
Give me some data to work on : onclick="alert(1)"
 It can be Cross site scripting attack
Give me some data to work on : alert(1)
seems safe
Give me some data to work on : <audio autoplay onplay=alert(1)><source src="validaudio.wav" type="audio/wav"></audio>
 It can be Cross site scripting attack
Give me som

Give me some data to work on : <rt onmousedown="alert(1)">test</rt>
 It can be Cross site scripting attack
Give me some data to work on : <s onblur=alert(1) tabindex=1 id=x></s><input autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <script draggable="true" ondragstart="alert(1)">test</script>
 It can be Cross site scripting attack
Give me some data to work on : <script oncut="alert(1)" contenteditable>test</script>
 It can be Cross site scripting attack
Give me some data to work on : <section onmouseup="alert(1)">test</section>
 It can be Cross site scripting attack
Give me some data to work on : <shadow id=x tabindex=1 onfocusin=alert(1)></shadow>
 It can be Cross site scripting attack
Give me some data to work on : <slot draggable="true" ondragstart="alert(1)">test</slot>
 It can be Cross site scripting attack
Give me some data to work on : <small onkeyup="alert(1)" contenteditable>test</small>
 It can be Cross site scripting attack
Give me some data 